In [2]:
import yaml
import pandas as pd
import numpy as np

import clipboard

from names import *
from styles import *

In [3]:
parameter_name_map = {
    'loc': '$\mu$',
    'sigma_c': '$\sigma_c$',
    'sigma_s': '$\sigma_s$',
    'scale': '$\sigma$',
    'sigma': '$\sigma$',
    'h': '$h$',
    'density': 'Density',
    'intensity': 'Intensity',
    'size': 'Kernel size'
}

type_name_map = {
    'const': 'Constant',
    'samples_num': 'Interpolation',
    'samples_step': 'Increments'
}

In [4]:
with open('data/optim_config.yaml') as f:
    data = yaml.safe_load(f)

params = []
keys = ['start', 'stop', 'num', 'step']
for f, par_dict in data.items():
    for p, config in par_dict.items():
        t = config['type']
        s_dict = config['params']
        s_vals = [s_dict.get(k, 0) for k in keys]
        ns = {
            'Type': type_name_map[t],
            'Filter': filter_name_map[f],
            'Parameter': parameter_name_map[p],
        }
        ns.update(s_dict)
        params.append(ns)

df = pd.DataFrame(params)
df.sort_values(['Type', 'Filter', 'Parameter'], inplace=True)
df = df.drop(columns='val')
df = df[df['Type'] != 'Constant']

dfa = df[df['Type'] == 'Increments']
dfa.drop(columns=['Type', 'num'], inplace=True)
dfb = df[df['Type'] == 'Interpolation']
dfb.drop(columns=['Type', 'step'], inplace=True)

/Users/Anton/anaconda3/envs/iaml/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
dfb

,Filter,Parameter,start,stop,num
4,Bilateral filter,$\sigma_c$,5.00,100.00,30.0
3,Bilateral filter,$\sigma_s$,1.00,20.00,30.0
19,Cauchy noise,$\sigma$,1.00,50.00,100.0
7,Comb,$\sigma$,1.00,50.00,20.0
6,Comb,$\sigma_c$,10.00,60.00,10.0
5,Comb,$\sigma_s$,1.00,20.00,10.0
10,Comb Reverse,$\sigma$,1.00,50.00,20.0
9,Comb Reverse,$\sigma_c$,10.00,60.00,10.0
8,Comb Reverse,$\sigma_s$,1.00,20.00,10.0
0,Gaussian filter,$\sigma$,0.00,50.00,100.0


In [6]:
levels = pd.MultiIndex.from_frame(dfa).to_frame()
levels = levels[[]]
#levels.reset_index(0, inplace=True)
levels

,,,,
Filter,Parameter,start,stop,step
Mean filter,Kernel size,1.0,51.0,2.0
Median filter,Kernel size,1.0,51.0,2.0


In [8]:
dfb.to_csv('out/param_table_a.csv')

In [24]:
levels.to_latex(
    'out/param_table_b.tex',
    multicolumn=True, 
    multirow=True, 
    decimal='.', 
    index=True, 
    escape=False,
    #float_format='{:2.2f}',
    na_rep='NA')

# Iris params

In [10]:
names = ['1x', '1.5x', '2x', '5x', '10x']
rows = []
for n in names:
    with open(f'data/iris/{n}.yaml') as f:
        d = yaml.safe_load(f)['filters']
        res = []
        for k, v in d.items():
            for (pk, pv) in v.items():
                res.append((filter_name_map[k], n, parameter_name_map[pk], pv))
        d = pd.DataFrame(res, columns=['Filter', 'Experiment', 'Parameter', 'Value'])
        #d = d.pivot(columns='experiment')
        
        rows.append(d)

frame = pd.concat(rows)

In [11]:
f = frame.pivot(columns='Experiment', index=['Filter', 'Parameter'])
f = f.reindex(names, level=1, axis=1)
f

Value                         \
Experiment                                1x       1.5x          2x   
Filter                Parameter                                       
Bilateral filter      $\sigma_c$   44.310345  54.137931   57.413793   
                      $\sigma_s$   16.724138  12.137931   10.827586   
Cauchy noise          $\sigma$      2.484848   6.939394    7.929293   
Comb                  $\sigma$      3.578947   6.157895    8.736842   
                      $\sigma_c$   32.222222  37.777778   37.777778   
                      $\sigma_s$   15.777778  15.777778   11.555556   
Comb Reverse          $\sigma$      6.157895  13.894737   16.473684   
                      $\sigma_c$   26.666667  26.666667   32.222222   
                      $\sigma_s$   13.666667  15.777778   17.888889   
Gaussian filter       $\sigma$      1.515152   2.525253    2.525253   
Gaussian noise        $\mu$         0.000000   0.000000    0.000000   
                      $\sigma$     21.212121  24.242424   36.363636   
Laplacian noise       $\sigma$     16.161616  16.161616   27.272727   
Mean filter           Kernel size   3.000000   3.000000    3.000000   
Median filter         Kernel size   5.000000   5.000000    5.000000   
Non-local means       $h$          10.101010  39.393939   42.424242   
Salt-and-pepper noise Density       0.640000   0.570000    0.360000   
                      Intensity    70.344828  79.137931  114.310345   
Snow noise            Density       0.069394   0.118889    0.247576   
Uniform noise         Intensity    60.606061  98.484848  106.060606   

                                                           
Experiment                                 5x         10x  
Filter                Parameter                            
Bilateral filter      $\sigma_c$    70.517241   77.068966  
                      $\sigma_s$    10.827586   10.172414  
Cauchy noise          $\sigma$      25.747475   39.111111  
Comb                  $\sigma$      11.315789   39.684211  
                      $\sigma_c$    54.444444   32.222222  
                      $\sigma_s$    15.777778    9.444444  
Comb Reverse          $\sigma$      31.947368   47.421053  
                      $\sigma_c$    48.888889   26.666667  
                      $\sigma_s$    17.888889   11.555556  
Gaussian filter       $\sigma$       2.525253    3.030303  
Gaussian noise        $\mu$          0.000000    0.000000  
                      $\sigma$      54.545455   87.878788  
Laplacian noise       $\sigma$      50.505051   77.777778  
Mean filter           Kernel size    5.000000    7.000000  
Median filter         Kernel size    7.000000    7.000000  
Non-local means       $h$           55.555556   55.555556  
Salt-and-pepper noise Density        0.500000    0.570000  
                      Intensity    184.655172  219.827586  
Snow noise            Density        0.366364    0.574242  
Uniform noise         Intensity    194.444444  250.000000

In [12]:
l = pd.MultiIndex.from_frame(f).to_frame()
l = l[[]]
l

"(Value, 1x)","(Value, 1.5x)","(Value, 2x)","(Value, 5x)","(Value, 10x)"
44.310345,54.137931,57.413793,70.517241,77.068966
16.724138,12.137931,10.827586,10.827586,10.172414
2.484848,6.939394,7.929293,25.747475,39.111111
3.578947,6.157895,8.736842,11.315789,39.684211
32.222222,37.777778,37.777778,54.444444,32.222222
15.777778,15.777778,11.555556,15.777778,9.444444
6.157895,13.894737,16.473684,31.947368,47.421053
26.666667,26.666667,32.222222,48.888889,26.666667
13.666667,15.777778,17.888889,17.888889,11.555556
1.515152,2.525253,2.525253,2.525253,3.030303


In [13]:
pd.options.display.float_format = '${:,.4f}'.format

In [14]:
f.to_latex(
    'out/iris-params.tex',
    multicolumn=True, 
    multirow=True, 
    decimal='.', 
    index=True, 
    longtable=False,
    escape=False,
    float_format=lambda x: '{:0.4f}'.format(x),
    na_rep='NA')

In [15]:
f.to_csv('out/iris-params.csv')

In [95]:
def get_table(filter_name):
    t = frame[[filter_name, 'experiment']].dropna().pivot(columns='experiment')
    t = t.reindex(names, level=1, axis=1)
    return t

In [57]:
ls = []
for (k, v) in filter_name_map.items():
    ls.append((k, get_table(v)))

In [59]:
for n, l in ls:
    l.to_latex(f'out/ptable/{n}.tex')